# Discrete Hopfield Network

In [903]:
import numpy as np
import csv

In [904]:
class DiscreteHopfieldNet:
    
    def __init__(self,filename):
        
        #Pass the filename containing fundamental memory
        self.filename = filename
        
        #Define the activation function
        self.activation = lambda x : self.signum(x)
        
        #Read the Fundamental Memory
        self.ReadMemoryFile()
        
        pass
    
    
    def signum(self,x):
    
        # Get a zero array of same size as x,to compare x with.
        y = np.zeros_like(x)

        # Get another zero array of same size as x, to store the truth value of (x >= y) 
        t = np.zeros_like(x)
        # To make t,have a min dimension of 1 (avoid O-D array)
        t = np.atleast_1d(t)

        #Compare x and y, and store result in t
        np.greater_equal(x , y , out = t , dtype = float)

        #Replace False value '0' with '-1', i.e, for all elements in x < 0 
        t = np.array([-1 if x== 0 else x for x in t], dtype = int)

        return t
    
    
        
        
        
    def ReadMemoryFile(self):
        
        # Open and Read Fundamental Memory from specified file
        f = open(self.filename,'r')
        
        #First line should contain no. of dimensions
        N =  f.readline()
        self.N = int(N[0])
             
        # Array to store Fundamental Memory    
        self.inputs = []
        
        for lines in f:
            
            datapoints = lines.split(',')
            self.inputs.append(datapoints)
        
        f.close()
        self.inputs = np.array(self.inputs,dtype =int)  
        
        # No. of Patterns is stored as self.P
        self.P = len(self.inputs) 
        
        # Variable to store the Nature of States
        self.NatureOfStates = [None] * (2 ** self.N)
        
        return self.inputs

    

  
    def WeightMatrix(self):
        
        #Form the weight matrix
        w = np.zeros((self.N,self.N) ,dtype = int)
        
        for x in self.inputs:
            x = x.reshape(self.N,1) 
            w += np.matmul(x,x.T)
        
        #Remove self links, i.e, the diagonal elements in WeightMatrix
        w = w - np.eye(self.N)*self.P
        
        #Scale by no. of dimensions
        w = w/self.N

        #Store Weights in self.W
        self.W = w
    
        return self.W
    
    
    def GeneratePresentStates(self):
        
        #Generate all possible PresentStates, in specified dimension
        
        n = self.N
        
        #Empty list to store PresentStates
        ps = []
        
        for i in range(2**n):
            
            #Get n-bit binary reprentation of states
            b = bin(i)[2:].zfill(n)  
            b = [int(i) for i in b]
            
            #Replace '0' with '-1'
            b = [-1 if j==0 else int(j) for j in b]

            #Append the state to PresentStates
            ps.append([b])
         
        #Store the PresentStates in self.ps
        self.ps = np.array(ps)
        
        return self.ps
    
    
    
    def GetNextState(self,presentState):
        
        #Get NextState for the passed PresentState, by SYNCHRONOUS UPDATE
        nextstate = np.matmul(self.W , presentState.T)
        nextstate = self.activation(nextstate)
        nextstate = np.array(nextstate, dtype = int)
        nextstate = nextstate.reshape(1,self.N)
        
        return nextstate
    
    
    def GenerateNextStates(self,PS):
        
        # Generates Nextstates for the passed array of PresentStates,PS
        NS = list( map(self.GetNextState , PS))
        NS = np.array(NS)        
        
        return NS
    
    
    
    def GetStationaryStates(self):
        
        #Empty list to hold Stationary states
        StatStates = []
        
        # Generate the PresentStates (as calling method 'AsyncUpdate' before this method 
        # might distort PresentStates)
        ps = self.GeneratePresentStates()
        
        #Generate the NextStates for the above PresentStates
        ns = self.GenerateNextStates(ps)
        
        #Check for Stationary States, i.e, wherever ps = ns
        for i in range( 2**self.N ):
            
            IsStatState = np.equal( ps[i] , ns[i] )
            
            if( np.all(IsStatState)):
                
                StatStates.append( ps[i] )
                
                #Store the nature of state in 'self.NatureOfStates'
                self.NatureOfStates[i] = "Stationary state"
                pass
            
            pass
                
        self.StatStates = np.array(StatStates)  
        

        return self.StatStates
    
    
    
    def GetLimitCycles(self):
        
        #Empty list to hold LimitCycles
        LimitCycles = []
        
        # Generate the PresentStates (as calling method 'AsyncUpdate' before this method 
        # might distort PresentStates)
        ps = self.GeneratePresentStates()
        
        #Generate the NextStates for the above found PresentStates
        ns = self.GenerateNextStates(ps)
        
        #Further generate NextStates for the NextStates above
        ns2     = self.GenerateNextStates(ns)
        
        #Check for LimitCycles i.e, wherever ps is same as ns2, but ns is not same as ns2
        for i in range( 2**self.N ):
            
            #Check if StationaryState
            IsStatState = np.equal( ps[i] , ns[i] )
            
            IsLimitCycle = np.equal( ps[i] , ns2[i] )
            
            
            # A stationary state should not be classified as a limit cycle
            if( np.all(IsLimitCycle) and not(np.all(IsStatState) )):
                LimitCycles.append( ps[i] )
                
                #Store the nature of state in 'self.NatureOfStates'
                self.NatureOfStates[i] = 'Limit Cycle'
                
        self.LimitCycles = np.array(LimitCycles)  
        
        
        return self.LimitCycles
    
    
    
    def GetEnergy(self,PS):
        
        # E = -1/2 * x * W * x.T ;    x ----> state
        
        W = self.W
        
        E = np.matmul( W , PS.T)
        E = -0.5 * np.matmul( PS , E)
        E = E[0][0]
        
        return E
    
    
    def GetAllEnergies(self):
                
        # Generate the PresentStates (as calling method 'AsyncUpdate' before this method 
        # might distort PresentStates)
        ps = self.GeneratePresentStates()
        
        E  = list( map(self.GetEnergy , ps) )
        
        self.E = E
        
        return E
    
    
    def AsyncUpdate(self,presentstate):
        
        # Making presentstate to be 1-D array for simplicity
        presentstate = presentstate.reshape(self.N)
        
        # i should run from 0 to (self.N - 1), (In Async.Update, a neuron is randomly chosen
        # and updated, here its a loop)
        i = 0
        
        while True:
            
            #Update the ith neuron,Continue until a stationary point is reached
            presentstate[i] = self.signum(np.matmul( A.W[i] , presentstate.T))
            
            #Check if Stationary State is reached 
            newstate = self.GetNextState(presentstate)
            IsStat = np.equal(newstate , presentstate)
            
            if( np.all(IsStat)):
                #Stationary State is reached
                break
            
            #Otherwise Continue loop
            if( i == (self.N-1) ):
                i = 0
            else:
                i += 1
             
            pass
        
                
        return list(presentstate)     
    
    
    def AsyncUpdateAll(self , ps):
        
        #Updates the array of states passed,ps Asynchronously
        return list(map(self.AsyncUpdate,ps))
        pass
    
            
            
    def DocumentStates(self):
        
        # Documentation of data
        
        self.ReadMemoryFile()
        self.WeightMatrix()
        self.GeneratePresentStates()
        self.ns = self.GenerateNextStates(self.ps)
        self.GetStationaryStates()
        self.GetLimitCycles()
        self.GetAllEnergies()
        
        fp = open('DiscreteHopfieldTable.csv','w')
        csvwriter = csv.writer(fp)
        csvwriter.writerows([ ['Fundamental Memory =' , self.inputs] , ['Weight Matrix =', self.W] \
                             ,[] ,[ None, 'SYNCHRONOUS UPDATE'] ,[]])
        
        fieldnames = ['StateIndex','PresentState','NextState','Energy','Nature']
        csvwriter = csv.DictWriter(fp,fieldnames=fieldnames)
        csvwriter.writeheader()
        
        N = self.N
        ps = self.ps.reshape(2**N , N)
        ns = self.ns.reshape(2**N , N)
        
        
        for i in range(2 ** self.N):
            csvwriter.writerow({'StateIndex': i+1,'PresentState': list(ps[i]), \
                                'NextState': list(ns[i]),'Energy':self.E[i], 'Nature': self.NatureOfStates[i] })
        
        
        ns_Async = self.AsyncUpdateAll(ps)
        #Regenerate PresentStates as method 'AsyncUpdate' distorts it
        ps = self.GeneratePresentStates().reshape(2**N , N)
        
        csvwriter = csv.writer(fp)
        csvwriter.writerows( [[None]]*4 + [['', 'ASYNCHRONOUS UPDATE']] + [[]])
        fieldnames = ['StateIndex','InitialState','FinalState']
        csvwriter = csv.DictWriter(fp,fieldnames=fieldnames)
        csvwriter.writeheader()
        
        for i in range(2 ** self.N):
            csvwriter.writerow({'StateIndex': i+1,'InitialState': list(ps[i]),'FinalState': list(ns_Async[i]) })
        

        fp.close()
        
        
        pass    
 

In [905]:
A = DiscreteHopfieldNet('fund_mem.csv')

In [906]:
A.WeightMatrix()

array([[ 0.        , -0.66666667,  0.66666667],
       [-0.66666667,  0.        ,  0.        ],
       [ 0.66666667,  0.        ,  0.        ]])

In [907]:
A.GeneratePresentStates()

array([[[-1, -1, -1]],

       [[-1, -1,  1]],

       [[-1,  1, -1]],

       [[-1,  1,  1]],

       [[ 1, -1, -1]],

       [[ 1, -1,  1]],

       [[ 1,  1, -1]],

       [[ 1,  1,  1]]])

In [908]:
A.DocumentStates()

In [943]:
# Probing the Hopfield Network

x = np.array([[-1,1,0] , [1,1,0] , [-1,0,1] , [1,-1,1]])
A.AsyncUpdateAll(x)

[[-1, 1, -1], [-1, 1, -1], [1, -1, 1], [1, -1, 1]]